Sortino Ratio does not punish volatility if it is in the upward direction
Set a threshold for volatility (typically zero)
sortino ratio = (mean asset return - risk free rate)/std. dev. of downside returns

In [23]:
import pandas_datareader as pdr
import datetime as dt
import numpy as np

In [24]:
#NOTE: we are going to get an extra day so we can drop it so we dont have a NaN on first day
#when you do the daily pct change
start = dt.datetime(2021, 2, 26)
end = dt.datetime(2022, 3, 1)

In [25]:
aapl = pdr.get_data_yahoo("AAPL", start, end)

In [26]:
aapl


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-02-26,124.849998,121.199997,122.589996,121.260002,164560400.0,120.543556
2021-03-01,127.930000,122.790001,123.750000,127.790001,116307900.0,127.034966
2021-03-02,128.720001,125.010002,128.410004,125.120003,102260900.0,124.380745
2021-03-03,125.709999,121.839996,124.809998,122.059998,112966300.0,121.338821
2021-03-04,123.599998,118.620003,121.750000,120.129997,178155000.0,119.420219
...,...,...,...,...,...,...
2022-02-23,166.149994,159.750000,165.539993,160.070007,90009200.0,160.070007
2022-02-24,162.850006,152.000000,152.580002,162.740005,141147500.0,162.740005
2022-02-25,165.119995,160.869995,163.839996,164.850006,91974200.0,164.850006


In [27]:
aapl['Daily Return'] = aapl['Adj Close'].pct_change(1)

In [28]:
aapl.dropna(inplace=True)

In [29]:
#Sharpe Ratio Daily Return Std Dev
aapl_std_dev_daily_return = aapl['Daily Return'].std()

In [30]:
def compute_sharpe_ratio(df, risk_free_rate=0):
    mean_return = df['Daily Return'].mean()
    std = df['Daily Return'].std()
    sharpe_ratio = (mean_return - risk_free_rate)/std
    
    annual_sharpe_ratio = sharpe_ratio * (np.sqrt(252))
    
    return annual_sharpe_ratio

In [31]:
sharpe_aapl = compute_sharpe_ratio(aapl)
sharpe_aapl

1.3339640384203142

In [32]:
def compute_sortino_ratio(df, risk_free_rate=0, threshold=0):
    mean_return = df['Daily Return'].mean()
    
    #Volatility---take into account threshold 
    #(note the last daily return is to just grab that column)
    downside = df[df['Daily Return'] < threshold]['Daily Return']
    std = downside.std()
    
    sortino_ratio = (mean_return - risk_free_rate)/std
    annual_sortino_ratio = sortino_ratio * (np.sqrt(252))
    
    return annual_sortino_ratio

In [33]:
sortino_aapl = compute_sortino_ratio(aapl)
sortino_aapl

2.2492363203998487

Remember the larger the Sharpe or Sortino the better. Apple shows a higher Sortino so it means it has more volatility upwards......Good